In [1]:
import sqlite3
from datetime import datetime

def display_tables(db_name="media_metadata1.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Medien-Metadaten-Tabelle anzeigen
    print("Medien-Metadaten:")
    cursor.execute("SELECT * FROM media_metadata")
    columns = [description[0] for description in cursor.description]
    print(" | ".join(columns))
    for row in cursor.fetchall():
        print(" | ".join(map(str, row)))

    print("\nMedien-Metriken-Historie:")
    # Medien-Metriken-Historie-Tabelle anzeigen
    cursor.execute("SELECT * FROM media_metrics_history")
    columns = [description[0] for description in cursor.description]
    print(" | ".join(columns))
    for row in cursor.fetchall():
        print(" | ".join(map(str, row)))

    conn.close()

if __name__ == "__main__":
    display_tables()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



19713 | 7169626331548830982 | 9087 | 530 | 8 | 20 | 1736631644
19714 | 7451949345135447318 | 798 | 84 | 0 | 2 | 1736631646
19715 | 7450851416714841366 | 1597 | 116 | 0 | 3 | 1736631646
19716 | 7450457149961981206 | 439 | 26 | 0 | 0 | 1736631646
19717 | 7445202826474654998 | 935 | 112 | 24 | 12 | 1736631646
19718 | 7444462672998862102 | 1397 | 55 | 6 | 8 | 1736631647
19719 | 7443590677310557462 | 9021 | 1561 | 63 | 142 | 1736631647
19720 | 7442077106282040598 | 10300 | 319 | 37 | 37 | 1736631647
19721 | 7434616208248360224 | 986 | 98 | 4 | 4 | 1736631647
19722 | 7434160817387343136 | 4676 | 163 | 1 | 9 | 1736631647
19723 | 7432435552319802657 | 794 | 31 | 1 | 0 | 1736631647
19724 | 7426297876231294241 | 281 | 32 | 0 | 1 | 1736631647
19725 | 7424496720672853280 | 874 | 110 | 18 | 14 | 1736631647
19726 | 7423106806232157472 | 794 | 154 | 1 | 7 | 1736631647
19727 | 7422769752411573536 | 11700 | 1319 | 27 | 164 | 1736631647
19728 | 7420797967109590304 | 852 | 180 | 7 | 15 | 1736631647
19729

In [4]:
import sqlite3
from datetime import datetime

def display_metrics_for_channel(channel_name="sichertdeutschland", db_name="media_metadata1.db"):
    # Verbinde mit der SQLite-Datenbank
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Video-IDs für den spezifischen Kanal abrufen
    cursor.execute("SELECT id FROM media_metadata WHERE channel = ?", (channel_name,))
    video_ids = [row[0] for row in cursor.fetchall()]

    if not video_ids:
        print(f"Kein Video gefunden für den Kanal: {channel_name}")
        return

    # Daten aus der media_metrics_history-Tabelle für die spezifischen Video-IDs abrufen
    cursor.execute("""
        SELECT video_id, view_count, like_count, repost_count, comment_count, timestamp
        FROM media_metrics_history
        WHERE video_id IN ({})
    """.format(','.join('?' * len(video_ids))), video_ids)
    data = cursor.fetchall()

    if not data:
        print(f"Keine Metriken gefunden für den Kanal: {channel_name}")
        return

    # Daten sortieren nach Timestamp
    metrics = [{
        "video_id": video_id,
        "timestamp": datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
        "view_count": view_count,
        "like_count": like_count,
        "repost_count": repost_count,
        "comment_count": comment_count
    } for video_id, view_count, like_count, repost_count, comment_count, timestamp in data]

    # Daten ausgeben
    print(f"Kanal: {channel_name}")
    for metric in sorted(metrics, key=lambda x: x["timestamp"]):
        print(f"Video ID: {metric['video_id']} | Zeit: {metric['timestamp']} | Views: {metric['view_count']} | "
              f"Likes: {metric['like_count']} | Reposts: {metric['repost_count']} | Kommentare: {metric['comment_count']}")

    conn.close()

if __name__ == "__main__":
    display_metrics_for_channel()

Kein Video gefunden für den Kanal: sichertdeutschland


In [3]:
import sqlite3
from datetime import datetime

def display_metrics_over_time(db_name="media_metadata1.db"):
    # Verbinde mit der SQLite-Datenbank
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Medien-Metadaten abrufen, um Kanalnamen zu erhalten
    cursor.execute("SELECT id, channel FROM media_metadata")
    channel_data = cursor.fetchall()
    channel_map = {video_id: channel for video_id, channel in channel_data}

    # Daten aus der media_metrics_history-Tabelle abrufen
    cursor.execute("SELECT video_id, view_count, like_count, repost_count, comment_count, timestamp FROM media_metrics_history")
    data = cursor.fetchall()

    # Daten gruppieren nach Video-ID und sortieren nach Timestamp
    metrics_by_channel = {}
    for video_id, view_count, like_count, repost_count, comment_count, timestamp in data:
        channel = channel_map.get(video_id, "Unbekannter Kanal")
        if channel not in metrics_by_channel:
            metrics_by_channel[channel] = []
        metrics_by_channel[channel].append({
            "timestamp": datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S'),
            "view_count": view_count,
            "like_count": like_count,
            "repost_count": repost_count,
            "comment_count": comment_count
        })

    # Daten ausgeben
    for channel, metrics in metrics_by_channel.items():
        print(f"\nKanal: {channel}")
        for metric in sorted(metrics, key=lambda x: x["timestamp"]):
            print(f"Zeit: {metric['timestamp']} | Views: {metric['view_count']} | Likes: {metric['like_count']} | Reposts: {metric['repost_count']} | Kommentare: {metric['comment_count']}")

    conn.close()

if __name__ == "__main__":
    display_metrics_over_time()


Kanal: GereonBollmann
Zeit: 2025-01-11 09:34:17 | Views: 867 | Likes: 111 | Reposts: 1 | Kommentare: 7
Zeit: 2025-01-11 09:34:17 | Views: 904 | Likes: 104 | Reposts: 0 | Kommentare: 5
Zeit: 2025-01-11 09:34:17 | Views: 882 | Likes: 103 | Reposts: 1 | Kommentare: 8
Zeit: 2025-01-11 09:34:17 | Views: 367 | Likes: 44 | Reposts: 5 | Kommentare: 6
Zeit: 2025-01-11 09:34:17 | Views: 1356 | Likes: 146 | Reposts: 1 | Kommentare: 7
Zeit: 2025-01-11 09:34:17 | Views: 843 | Likes: 112 | Reposts: 1 | Kommentare: 7
Zeit: 2025-01-11 09:34:17 | Views: 1471 | Likes: 164 | Reposts: 0 | Kommentare: 16
Zeit: 2025-01-11 09:34:17 | Views: 1062 | Likes: 189 | Reposts: 4 | Kommentare: 8
Zeit: 2025-01-11 09:34:17 | Views: 2672 | Likes: 422 | Reposts: 17 | Kommentare: 47
Zeit: 2025-01-11 09:34:17 | Views: 946 | Likes: 139 | Reposts: 3 | Kommentare: 7
Zeit: 2025-01-11 09:34:17 | Views: 806 | Likes: 89 | Reposts: 1 | Kommentare: 5
Zeit: 2025-01-11 09:34:17 | Views: 922 | Likes: 61 | Reposts: 5 | Kommentare: 5
Z